In [1]:
print("hello")

hello


In [6]:
""" istall the driver if does not exist """
# !pip install cassandra-driver

' istall the driver if does not exist '

In [7]:
""" Create the keyspace and created_users table for data loading if it does not exist """

# from cassandra.cluster import Cluster

# cluster = Cluster(['cassandra'])  # Update with your Cassandra host
# session = cluster.connect()

# # Create the keyspace
# keyspace_query = """
# CREATE KEYSPACE IF NOT EXISTS spark_streams 
# WITH replication = {'class': 'SimpleStrategy', 'replication_factor': 1};
# """
# session.execute(keyspace_query)
# print("Keyspace 'spark_streams' created (if not exists).")

# # Use the keyspace
# session.set_keyspace('spark_streams')

# # Create the table
# table_query = """
# CREATE TABLE IF NOT EXISTS created_users (
#     first_name text,
#     last_name text,
#     gender text,
#     address text,
#     postcode text,
#     email text,
#     username text,
#     dob text,
#     registered text,
#     phone text,
#     picture text,
#     PRIMARY KEY ((first_name, last_name), email)
# );
# """
# session.execute(table_query)
# print("Table 'created_users' created (if not exists).")

# # Close the connection
# cluster.shutdown()

' Create the keyspace and created_users table for data loading if it does not exist '

In [8]:
# Create the Spark Session
from pyspark.sql import SparkSession

spark = (
    SparkSession 
    .builder 
    .appName("Streaming from Kafka") 
    .config("spark.streaming.stopGracefullyOnShutdown", True) 
    .config('spark.jars.packages', 'org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0')
    .config("spark.sql.shuffle.partitions", 4)
    .master("local[*]") 
    .getOrCreate()
)

spark

KeyboardInterrupt: 

In [11]:
# Create the kafka_df to read from kafka

kafka_df = (
    spark
    .readStream
    .format("kafka")
    .option("kafka.bootstrap.servers", "ed-kafka:29092")
    .option("subscribe", "users_created")
    .option("startingOffsets", "earliest")
    .load()
)

In [15]:
# View schema for raw kafka_df
kafka_df.printSchema()
#kafka_df.show()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [16]:
# Parse value from binay to string into kafka_json_df
from pyspark.sql.functions import expr

kafka_json_df = kafka_df.withColumn("value", expr("cast(value as string)"))

In [17]:
from pyspark.sql.types import StringType, StructField, StructType, ArrayType, LongType

json_schema = StructType([
        StructField("first_name", StringType(), True),
        StructField("last_name", StringType(), True),
        StructField("gender", StringType(), True),
        StructField("address", StringType(), True),
        StructField("postcode", StringType(), True),
        StructField("email", StringType(), True),
        StructField("username", StringType(), True),
        StructField("dob", StringType(), True),
        StructField("registered", StringType(), True),
        StructField("phone", StringType(), True),
        StructField("picture", StringType(), True)
    ])

In [18]:
# Apply the schema to payload to read the data
from pyspark.sql.functions import from_json,col

streaming_df = kafka_json_df.withColumn("values_json", from_json(col("value"), json_schema)).selectExpr("values_json.*")

In [19]:
# To the schema of the data, place a sample json file and change readStream to read 
streaming_df.printSchema()
#streaming_df.show(truncate=False)

root
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- address: string (nullable = true)
 |-- postcode: string (nullable = true)
 |-- email: string (nullable = true)
 |-- username: string (nullable = true)
 |-- dob: string (nullable = true)
 |-- registered: string (nullable = true)
 |-- phone: string (nullable = true)
 |-- picture: string (nullable = true)



In [ ]:
(streaming_df
 .writeStream
 .format("console")
 .outputMode("append")
 .option("checkpointLocation", "checkpoint_dir_kafka")
 .start()
 .awaitTermination())